In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Nullable{RigidBodyDynamics.SpatialInertia{Float64}}())

In [3]:
J = rand(Mat{3, 3, Float64}); J = J * J'
c = rand(Vec{3, Float64})
m = rand()
inertia1 = SpatialInertia(CartesianFrame3D("body1"), J, c, m)
body1 = RigidBody(inertia1);
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = Transform3D(joint1.frameBefore, world.frame, nquatrand(), rand(Vec{3, Float64}))
body1ToJoint1 = Transform3D(inertia1.frame, joint1.frameAfter, nquatrand(), rand(Vec{3, Float64}))
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
inertia2 = SpatialInertia(joint2.frameAfter, J, c, m)
body2 = RigidBody(inertia2)
joint2ToBody1 = Transform3D(joint2.frameBefore, body1.frame, nquatrand(), rand(Vec{3, Float64}))
attach!(mechanism, body1, joint2, joint2ToBody1, body2);

In [19]:
x = MechanismState{Float64}(mechanism);
zero_configuration!(x)

In [20]:
num_positions(x)

8

In [21]:
num_velocities(x)

7

In [22]:
cache = FrameCache(mechanism, x);

In [23]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.5207290356705264 + 0.5282298047921005im - 0.1187445401910118jm - 0.6600865692572214km,FixedSizeArrays.Vec{3,Float64}((0.23690563515156726,0.32519314925632203,0.3239675744071018)))

In [24]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((1.8416236798533847,-0.22827264712124196,1.1273141641179958)))

In [25]:
mass(mechanism)

0.012438020059316823

In [26]:
sum([isnull(body.inertia) ? 0. : get(body.inertia).mass for body in bodies(mechanism)])

0.012438020059316823

In [27]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((1.5272119741049686,-0.19415011800116022,1.5195706679501548)))